# Lab 3 - Enter the Model Context Protocol!

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
load_dotenv(override=True)
MODEL_NAME = "gpt-4.1-mini"

In [ ]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
    tools = await fetch.session.list_tools()

tools.tools

In [ ]:
playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as playwright:
    playwright_tools = await playwright.session.list_tools()

playwright_tools.tools

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
    file_tools = await filesystem.session.list_tools()

file_tools.tools

In [8]:
instructions = "You use your tools to navigate the web and write summaries to a file"

input = "Bring up a browser, visit 3 different news sites, click on at least 1 story on each site, and write a summary of the news in markdown to a file news.md in the sandbox directory"

In [ ]:
with trace("News"):
    async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
        async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
            agent = Agent(name="News", instructions=instructions, model=MODEL_NAME, mcp_servers=[fetch, filesystem])
            result = await Runner.run(agent, input)
            print(result.final_output)

In [ ]:
with trace("News"):
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as playwright:
        async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
            agent = Agent(name="News", instructions=instructions, model=MODEL_NAME, mcp_servers=[playwright, filesystem])
            result = await Runner.run(agent, input)
            print(result.final_output)